# Session 3-- Graphing and manipulating data

## Session 3 is composed of 2 parts

- Module 5: Graphing and mapping
    - [Notes](https://github.com/corybaird/SPP_Data_Seminar/blob/main/R/Session_3/Module_5/Module_5_GraphMap.ipynb)
    - [Video](https://www.youtube.com/watch?v=5hrFI-_Rtqk)

- Module 6: Data manipulation and panel regressions
   - [Notes](https://github.com/corybaird/SPP_Data_Seminar/blob/main/R/Session_3/Module_6/Module_6_ManipulateModel.ipynb)
   - [Video](https://youtu.be/xVo8zsi6DrI)

## A.1 Import Libraries

In [2]:
library(dplyr)

# Step 1: install
#install.packages('WDI')
# Step 2: load functions
library(WDI)

## A.2 Import data

In [3]:
df_wdi = WDI(
country = "all",
indicator = c("NY.GDP.PCAP.KD", #GDP
              "SP.POP.DPND" # Age dependency
             ),
start = 1980,
end = 2020,
)

In [4]:
df_wdi = df_wdi %>% 
rename('GDP' = 'NY.GDP.PCAP.KD',
      'Age_dep_ratio' = 'SP.POP.DPND')

In [5]:
df_wdi = df_wdi %>% select(country, year, GDP, Age_dep_ratio)

# 1. Data manipulation

## 1.A Data types

In [6]:
df_wdi %>% 
str()

'data.frame':	10906 obs. of  4 variables:
 $ country      : chr  "Arab World" "Arab World" "Arab World" "Arab World" ...
 $ year         : int  1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 ...
 $ GDP          : num  5263 5255 4634 4185 4114 ...
  ..- attr(*, "label")= chr "GDP per capita (constant 2010 US$)"
 $ Age_dep_ratio: num  92.3 92 91.5 91 90.4 ...
  ..- attr(*, "label")= chr "Age dependency ratio (% of working-age population)"


## 1.B Summary stats

In [7]:
df_wdi %>% 
summary()

   country               year           GDP           Age_dep_ratio   
 Length:10906       Min.   :1980   Min.   :   164.3   Min.   : 15.74  
 Class :character   1st Qu.:1990   1st Qu.:  1410.4   1st Qu.: 50.63  
 Mode  :character   Median :2000   Median :  3977.6   Median : 62.32  
                    Mean   :2000   Mean   : 12105.2   Mean   : 67.05  
                    3rd Qu.:2010   3rd Qu.: 13606.1   3rd Qu.: 84.81  
                    Max.   :2020   Max.   :209224.5   Max.   :117.88  
                                   NA's   :1585       NA's   :1006    

## 1.1 Check for NA

In [8]:
df_wdi %>% 
is.na() %>% 
any()

[1] TRUE

### 1.1.1 Drop na


In [9]:
df_wdi %>% count()

n
10906


In [10]:
df_wdi  %>% 
na.omit()  %>% count()

n
8841


## 1.2 Create dummy variables

### 1.2.A Filter 

In [11]:
df_wdi_2020 = df_wdi %>% filter(year== 2020)
df_wdi_2020 %>% tail(2)

,country,year,GDP,Age_dep_ratio
265,IDA & IBRD total,2020,4822.537,54.63396
266,Zimbabwe,2020,1058.846,81.57150


### 1.2.1 Add dummy for high-income countries

In [12]:
df_wdi_2020 = df_wdi_2020 %>% 
mutate(high_inc = as.numeric(GDP>15000)) 

df_wdi_2020 %>% head(2)

country,year,GDP,Age_dep_ratio,high_inc
Arab World,2020,6043.964,61.01996,0
World,2020,10565.499,54.56867,0


In [13]:
df_wdi_2020 %>% summary()

   country               year           GDP           Age_dep_ratio   
 Length:266         Min.   :2020   Min.   :   202.4   Min.   : 18.10  
 Class :character   1st Qu.:2020   1st Qu.:  1785.2   1st Qu.: 49.01  
 Mode  :character   Median :2020   Median :  5371.0   Median : 55.18  
                    Mean   :2020   Mean   : 12747.3   Mean   : 58.93  
                    3rd Qu.:2020   3rd Qu.: 13878.9   3rd Qu.: 68.59  
                    Max.   :2020   Max.   :107458.0   Max.   :109.50  
                                   NA's   :43         NA's   :25      
    high_inc     
 Min.   :0.0000  
 1st Qu.:0.0000  
 Median :0.0000  
 Mean   :0.2466  
 3rd Qu.:0.0000  
 Max.   :1.0000  
 NA's   :43      

### 1.2.2 Omit NAs for only one column

In [14]:
df_wdi_2020 = df_wdi_2020 %>% 
filter(!is.na(GDP))

df_wdi_2020 %>% summary()

   country               year           GDP           Age_dep_ratio   
 Length:223         Min.   :2020   Min.   :   202.4   Min.   : 18.10  
 Class :character   1st Qu.:2020   1st Qu.:  1785.2   1st Qu.: 49.24  
 Mode  :character   Median :2020   Median :  5371.0   Median : 55.43  
                    Mean   :2020   Mean   : 12747.3   Mean   : 59.47  
                    3rd Qu.:2020   3rd Qu.: 13878.9   3rd Qu.: 69.56  
                    Max.   :2020   Max.   :107458.0   Max.   :109.50  
                                                      NA's   :5       
    high_inc     
 Min.   :0.0000  
 1st Qu.:0.0000  
 Median :0.0000  
 Mean   :0.2466  
 3rd Qu.:0.0000  
 Max.   :1.0000  
                 

### 1.3 Use dummy with summary stats

In [15]:
df_wdi_2020 %>% 
group_by(high_inc) %>% 
summarise(gdp_mean = mean(GDP))

high_inc,gdp_mean
0,4437.55
1,38129.82


## 1.3 Mapping values

In [16]:
df_wdi_2020  %>% 
mutate(highinc_dummy_factor = recode(high_inc, '0'='Low', '1'='High')) %>% 
tail(2)

,country,year,GDP,Age_dep_ratio,high_inc,highinc_dummy_factor
222,IDA & IBRD total,2020,4822.537,54.63396,0,Low
223,Zimbabwe,2020,1058.846,81.57150,0,Low


## 1.4 Cut-off dummies

In [17]:
cutoffs = c(seq(0, 60000, by = 10000))
cutoffs

[1]     0 10000 20000 30000 40000 50000 60000

In [18]:
df_wdi_2020 = df_wdi_2020  %>%  
mutate(cut_variable = cut(df_wdi_2020$GDP, cutoffs, include.lowest=TRUE))

df_wdi_2020 %>% head(2)

country,year,GDP,Age_dep_ratio,high_inc,cut_variable
Arab World,2020,6043.964,61.01996,0,"[0,1e+04]"
World,2020,10565.499,54.56867,0,"(1e+04,2e+04]"


In [19]:
df_wdi_2020 %>% 
group_by(cut_variable) %>% 
summarise(mean_gdp = mean(GDP))

cut_variable,mean_gdp
"[0,1e+04]",3684.49
"(1e+04,2e+04]",14247.69
"(2e+04,3e+04]",24778.02
"(3e+04,4e+04]",35272.63
"(4e+04,5e+04]",44606.33
"(5e+04,6e+04]",55529.08
NA,84730.44


## 1.5 Filter list

In [20]:
country_list = c('Albania', 'Italy', 'France', 'Belgium')
df_wdi_2020  %>% filter(country %in% country_list)

country,year,GDP,Age_dep_ratio,high_inc,cut_variable
Albania,2020,5064.062,46.93011,0,"[0,1e+04]"
Belgium,2020,44361.248,56.95681,1,"(4e+04,5e+04]"
France,2020,40520.652,62.35641,1,"(4e+04,5e+04]"
Italy,2020,32901.883,56.95922,1,"(3e+04,4e+04]"


## 1.6 Case when

In [21]:
spanish_country_list = c('Spain', 'Argentina', 'Mexico','Chile')

df_wdi_2020 %>% 
mutate(language = case_when(country=='Spain'~'Spanish', 
                           country=='Italy' ~ 'Italian', 
                           country=='United Kingdom'~'English')) %>% na.omit()

,country,year,GDP,Age_dep_ratio,high_inc,cut_variable,language
56,Spain,2020,29600.35,52.38578,1,"(2e+04,3e+04]",Spanish
64,United Kingdom,2020,39102.90,57.06137,1,"(3e+04,4e+04]",English
87,Italy,2020,32901.88,56.95922,1,"(3e+04,4e+04]",Italian


## 1.7 Count column

In [22]:
df_wdi_2020  %>% count(high_inc)

high_inc,n
0,168
1,55


## 1.8 Export data

In [23]:
df_wdi_2020 %>% write.csv('Output/wdi_data.csv')

# 2. Merging

## 2.A Import data

In [24]:
library(gapminder)
gapminder %>% head(2)

country,continent,year,lifeExp,pop,gdpPercap
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530


## 2.B Filter data

In [25]:
gapminder_2007 = gapminder %>% filter(year==2007)

## 2.1 Merge rows

In [26]:
df_1 = gapminder_2007[1:3, ]
df_1

country,continent,year,lifeExp,pop,gdpPercap
Afghanistan,Asia,2007,43.828,31889923,974.5803
Albania,Europe,2007,76.423,3600523,5937.0295
Algeria,Africa,2007,72.301,33333216,6223.3675


In [27]:
df_2 = gapminder_2007[5:7, ]
df_2

country,continent,year,lifeExp,pop,gdpPercap
Argentina,Americas,2007,75.320,40301927,12779.38
Australia,Oceania,2007,81.235,20434176,34435.37
Austria,Europe,2007,79.829,8199783,36126.49


In [28]:
rbind(df_1, df_2)

country,continent,year,lifeExp,pop,gdpPercap
Afghanistan,Asia,2007,43.828,31889923,974.5803
Albania,Europe,2007,76.423,3600523,5937.0295
Algeria,Africa,2007,72.301,33333216,6223.3675
Argentina,Americas,2007,75.320,40301927,12779.3796
Australia,Oceania,2007,81.235,20434176,34435.3674
Austria,Europe,2007,79.829,8199783,36126.4927


## 2.2 Merge columns

In [29]:
df_1 = gapminder_2007[1:3, c('year','lifeExp')]
df_1

year,lifeExp
2007,43.828
2007,76.423
2007,72.301


In [30]:
df_2 = gapminder_2007[5:7, c('continent','country')]
df_2

continent,country
Americas,Argentina
Oceania,Australia
Europe,Austria


In [31]:
cbind(df_1, df_2)

year,lifeExp,continent,country
2007,43.828,Americas,Argentina
2007,76.423,Oceania,Australia
2007,72.301,Europe,Austria


## 2.3 Merge rows and columns

### 2.3.1 Case data

In [32]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
nyt_cases_df = read.csv(url)
nyt_cases_df  %>% head(2)

date,county,state,fips,cases,deaths
2020-01-21,Snohomish,Washington,53061,1,0
2020-01-22,Snohomish,Washington,53061,1,0


### 2.3.2 Mask data

In [33]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv'
nyt_mask_df = read.csv(url)
nyt_mask_df %>% head(2)

COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
1001,0.053,0.074,0.134,0.295,0.444
1003,0.083,0.059,0.098,0.323,0.436


### 2.3.3 Merge

#### 5.3.3.1 For the column merge on make sure the name is the same in both data sets

In [34]:
library(dplyr)

In [35]:
nyt_mask_df = nyt_mask_df %>% rename('fips' = 'COUNTYFP' )
nyt_mask_df %>% names()

[1] "fips"       "NEVER"      "RARELY"     "SOMETIMES"  "FREQUENTLY"
[6] "ALWAYS"

In [36]:
merge(nyt_cases_df, nyt_mask_df, by='fips') %>% head(3)

fips,date,county,state,cases,deaths,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
1001,2020-05-15,Autauga,Alabama,103,4,0.053,0.074,0.134,0.295,0.444
1001,2021-04-02,Autauga,Alabama,6606,99,0.053,0.074,0.134,0.295,0.444
1001,2020-04-05,Autauga,Alabama,12,0,0.053,0.074,0.134,0.295,0.444


# 3. Other models

## 3.1 Fixed effects

In [37]:
library(plm)


Attaching package: ‘plm’

The following objects are masked from ‘package:dplyr’:

    between, lag, lead



In [38]:
data("Grunfeld", package="plm")
Grunfeld  %>% head(3)

firm,year,inv,value,capital
1,1935,317.6,3078.5,2.8
1,1936,391.8,4661.7,52.6
1,1937,410.6,5387.1,156.9


### 3.1.1 Fixed effects: Individual fixed effects


In [39]:
grun.fe <- plm(inv~value+capital, 
               index=c('firm','year'), 
               data = Grunfeld, 
               model = "within")

summary(grun.fe)

Oneway (individual) effect Within Model

Call:
plm(formula = inv ~ value + capital, data = Grunfeld, model = "within", 
    index = c("firm", "year"))

Balanced Panel: n = 10, T = 20, N = 200

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-184.00857  -17.64316    0.56337   19.19222  250.70974 

Coefficients:
        Estimate Std. Error t-value  Pr(>|t|)    
value   0.110124   0.011857  9.2879 < 2.2e-16 ***
capital 0.310065   0.017355 17.8666 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    2244400
Residual Sum of Squares: 523480
R-Squared:      0.76676
Adj. R-Squared: 0.75311
F-statistic: 309.014 on 2 and 188 DF, p-value: < 2.22e-16

### 3.1.2 Fixed effects: Individual AND time fixed effects

In [40]:
#Method 1: factor(year)
grun.fe <- plm(inv~value+capital+factor(year), index=c('firm','year'), data = Grunfeld, model = "within")

#Method 2: effect='twoways'
grun.fe <- plm(inv~value+capital, index=c('firm','year'), data = Grunfeld, model = "within", effect='twoways')
summary(grun.fe)$coefficients

,Estimate,Std. Error,t-value,Pr(>|t|)
value,0.1177159,0.01375128,8.560354,6.652575e-15
capital,0.3579163,0.02271901,15.754043,5.453066e-35


### 3.1.3 Standard errors
### 3.1.3.1 Baseline standard error

In [41]:
summary(grun.fe)$coefficients

,Estimate,Std. Error,t-value,Pr(>|t|)
value,0.1177159,0.01375128,8.560354,6.652575e-15
capital,0.3579163,0.02271901,15.754043,5.453066e-35


### 3.1.1.2 Clustered standard error

In [42]:
library(lmtest)

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



In [43]:
coeftest(grun.fe, vcovHC(grun.fe, type = 'HC0', cluster = c('group','time')))


t test of coefficients:

        Estimate Std. Error t value  Pr(>|t|)    
value   0.117716   0.009712  12.121 < 2.2e-16 ***
capital 0.357916   0.042931   8.337 2.552e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## 3.2 Random effects

In [44]:
grun.re <- plm(inv~value+capital,
               index=c('firm','year'), 
               data = Grunfeld, 
               model = "random")

summary(grun.re)$coefficients

,Estimate,Std. Error,z-value,Pr(>|z|)
(Intercept),-57.8344149,28.89893526,-2.001265,4.536389e-02
value,0.1097812,0.01049266,10.462658,1.282075e-25
capital,0.3081130,0.01718047,17.933910,6.410879e-72


### 3.2.1 Robust standard errors

In [45]:
coeftest(grun.re, vcovHC(grun.re, type = 'HC0', cluster = c('group','time')))


t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept) -57.834415  23.449626 -2.4663   0.01451 *  
value         0.109781   0.012984  8.4551 6.186e-15 ***
capital       0.308113   0.051889  5.9379 1.284e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 3.Z More info on clustered standard errors
- [Tutorial](http://rforpublichealth.blogspot.com/2014/10/easy-clustered-standard-errors-in-r.html)

- [Code](https://stackoverflow.com/questions/33155638/clustered-standard-errors-in-r-using-plm-with-fixed-effects)

- [Compare R and STATA clustered SE](http://www.richard-bluhm.com/clustered-ses-in-r-and-stata-2/)


## 3.3 Logistic regression

In [46]:
mtcars %>% head(2)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21,6,160,110,3.9,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4


In [47]:
log_reg = glm(am~ cyl+disp+drat+mpg, family='binomial', mtcars)
summary(log_reg)


Call:
glm(formula = am ~ cyl + disp + drat + mpg, family = "binomial", 
    data = mtcars)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.81690  -0.34097  -0.04864   0.17467   1.74918  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)  
(Intercept) -45.47048   22.84741  -1.990   0.0466 *
cyl           2.36515    1.33387   1.773   0.0762 .
disp         -0.01922    0.01739  -1.105   0.2692  
drat          6.22298    2.91299   2.136   0.0327 *
mpg           0.60124    0.47441   1.267   0.2050  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 43.230  on 31  degrees of freedom
Residual deviance: 15.676  on 27  degrees of freedom
AIC: 25.676

Number of Fisher Scoring iterations: 7


## 3.4 Machine learning: Supervised learning

In [48]:
# Step 1
#install.packages('caret')

# Step 2
library(caret)

Loading required package: lattice
Loading required package: ggplot2


In [49]:
data(Sacramento)
house_df = Sacramento %>% as_tibble()
house_df  = house_df %>% mutate(ID = row_number())
house_df %>% head(3)

city,zip,beds,baths,sqft,type,price,latitude,longitude,ID
SACRAMENTO,z95838,2,1,836,Residential,59222,38.63191,-121.4349,1
SACRAMENTO,z95823,3,1,1167,Residential,68212,38.47890,-121.4310,2
SACRAMENTO,z95815,2,1,796,Residential,68880,38.61830,-121.4438,3


### 3.4.1. Baseline regression

In [50]:
model = lm(price ~ beds+baths+sqft, data=house_df)

#### 3.4.1.1 Evaluate model

In [51]:
rmse = function(acutal, predicted){
    error = predicted - acutal
    rmse = sqrt(mean(error^2))
    return(rmse)
}

#### 3.4.1.2 Actual data=price , Predicted data=fitted.values

In [52]:
rmse(house_df$price, model$fitted.values)

[1] 82436.61

### 3.1.2 Train-test regression

In [53]:
set.seed(42)

train = house_df  %>% sample_frac(.8)
train %>% head(2)

city,zip,beds,baths,sqft,type,price,latitude,longitude,ID
SACRAMENTO,z95822,2,2,990,Residential,66500,38.49350,-121.4753,561
CARMICHAEL,z95608,2,2,1598,Residential,484000,38.60275,-121.3293,321


In [54]:
test = house_df[-train$ID,]
test %>% head(2)

city,zip,beds,baths,sqft,type,price,latitude,longitude,ID
SACRAMENTO,z95824,2,1,797,Residential,81900,38.51947,-121.4358,5
SACRAMENTO,z95841,3,1,1122,Condo,89921,38.66260,-121.3278,6


In [55]:
model = lm(price ~ beds+baths+sqft, data=train)

p = predict(model, test)

rmse(p, test$price)

[1] 86595.17

### 3.1.3 Cross validation

- More information on the following [blog post](https://medium.com/the-owl/k-fold-cross-validation-in-keras-3ec4a3a00538)

<img src= https://miro.medium.com/max/601/1*PdwlCactbJf8F8C7sP-3gw.png>

In [56]:
model = train(
  price ~ beds+baths+sqft, 
    data=house_df,
  method = "lm",
  trControl = trainControl(
    method = "cv", 
    number = 10,
    verboseIter = FALSE
  )
)
model

Linear Regression 

932 samples
  3 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 840, 839, 837, 838, 840, 838, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  82523.46  0.6070828  60606.13

Tuning parameter 'intercept' was held constant at a value of TRUE